In [165]:
'''
1. Do the number of songs for each year/artist have to be roughly equal for training?
2. Embed words given the line that they are in? independent from rest of lyrics
3. Remove or keep/use section tags, e.g [Chorus]
'''
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import requests, json
from bs4 import BeautifulSoup
import re

url = 'https://en.wikipedia.org/wiki/List_of_hip_hop_musicians'
page = requests.get(url)
aList = []
pattern = r"(<li><a href=\"\/wiki\/)(.*?)(\" title=\")(.*?)(\">)(.*?)(<\/a><\/li>)"
for x in re.finditer(pattern, page.text, re.S):
    artist = x.group(4)
    if artist == "List of hip hop groups":
        break
    if "(" in artist:
        artist = artist.split("(")[0][:-1]
        aList.append(artist)
    else:
        for char in artist:
            if not char.isalnum() and char != " ":
                artist = artist.replace(char,"")
        aList.append(artist)
#print(aList)

In [166]:
GENIUS_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = "Enter Token Here"
HEADERS = {"Authorization": "Bearer {}".format(CLIENT_ACCESS_TOKEN)}

def gen_get(path, params = None, headers = None):

    url = '/'.join([GENIUS_URL, path])
    
    response = requests.get(url=url, params=params, headers=HEADERS)
    response.raise_for_status()

    return response.json()

def get_hits(artist_id):

    curr = 1
    next = True
    songs = []

    while next:

        path = "artists/{}/songs/".format(artist_id)
        params = {'page': curr}
        data = gen_get(path=path, params=params, headers = HEADERS)
        pages = data['response']['songs']
        if pages:
            songs += pages
            curr += 1
        else:
            next = False

    songs = [song["id"] for song in songs if song["primary_artist"]["id"] == artist_id]
    return songs

# Populate
full = []

# Truncated aList for testing
for name in aList[:1]:
    find_id = gen_get("search", {'q': name})
    for hit in find_id["response"]["hits"]:
        if hit["result"]["primary_artist"]["name"] == name:
            artist_id = hit["result"]["primary_artist"]["id"]
            break
    song_ids = get_hits(artist_id)
            
    full.append(song_ids)
#print(full)

In [167]:
d = {}

for artist_songs in full:
    # Truncated song list for testing
    for song in artist_songs[:3]:
        api_url = GENIUS_URL + "/songs/" + str(song)
        response = requests.get(api_url, headers=HEADERS)
        json = response.json()
        # value: [song title, artist name, artist id, release year]
        song_title = json["response"]["song"]["title"]
        artist_name = json["response"]["song"]["primary_artist"]["name"]
        artist_id = json["response"]["song"]["primary_artist"]["id"]
        release_date = json["response"]["song"]["release_date"]
        # Check if release date is available
        if release_date:
            year = release_date[:4]
            
            # Get lyrics
            song_url = json["response"]["song"]["url"]
            request = requests.get(song_url, headers=HEADERS)
            page = BeautifulSoup(request.text, "html.parser")
            [l.extract() for l in page('script')]
            lyrics = page.find("div", class_="lyrics").get_text()
            
            d[song] = [song_title, artist_name, artist_id, year, lyrics]
#print(d)